# Import Modules

In [1]:
import os
print(os.getcwd())
import sys

import json

from pandas import DataFrame

# from pymatgen import MPRester, Structure

from mpcontribs.client import load_client

/home/raulf2012/Dropbox/01_norskov/00_git_repos/PROJ_IrOx_Active_Learning_OER/CatHub_MPContribs_upload/MPContribs_upload


In [2]:
from pymatgen.io.ase import AseAtomsAdaptor

# Read IrOx DFT Data

In [3]:
# #########################################################
import pickle; import os
path_i = os.path.join(
    os.environ["PROJ_irox"],
    "workflow/ml_modelling/processing_bulk_dft/creating_final_dataset_for_upload",
    "out_data/df_dft_final_no_dupl.pickle")
with open(path_i, "rb") as fle:
    df_dft = pickle.load(fle)
# #########################################################

df_dft = df_dft.drop(columns=["id", "form_e_chris", "path", "source"])

In [4]:
df_dft = df_dft.sample(n=10)

# df_dft.head()

# MPContribs

In [5]:
api_key = "EKw2ikaWO5TtUTev5tQSMaZf5oEUyvch"

project = 'active_learned_irox_polymorphs'

client = load_client(api_key)

# print(dir(client))
# print(dir(client.projects))

# Deleting all data to start over

In [6]:
deleted = client.contributions.delete_entries(project=project).result()

print(deleted['count'], 'contribution(s) deleted')

deleted

10 contribution(s) deleted


{'count': 10, 'has_more': False, 'total_count': 10, 'total_pages': 1}

# Create Project (Once)

In [7]:
is_public = True
info = {
    'project': project,
    'is_public': is_public,
    'title': 'Active Learned IrOx Polymorphs',
    'owner': 'raulf2012@gmail.com',
    'authors': 'R. Flores, W. Kirsten',
    'description': ' '.join('''
        This is a TEMP description here,
        nothing really to say right now
    '''.replace('\n', '').split()),
    'urls': {
        # 'JARVIS': 'http://www.ctcms.nist.gov/~knc6/JVASP.html',
        # 'SciRep': 'https://doi.org/10.1038/s41598-017-05402-0',
        'GitHub': 'https://github.com/'
    }
}
if False:
    client.projects.create_entry(project=info).result()

In [8]:
all_data = client.projects.get_entry(pk=project, _fields=['_all']).result()

# all_data

# Adding new data rows

In [13]:
contributions = dict()
for ind_i, row_i in df_dft.iterrows():
    stoich = row_i.stoich
    dH = row_i.dH

    # #####################################################
    dH = str(dH) + " eV"

    # #####################################################
    stoich_to_form_dict = {
        "AB2": "IrO2",
        "AB3": "IrO3",
        }

    formula = stoich_to_form_dict.get(stoich, stoich)
    print(formula)

    contributions[ind_i] = dict(
        contrib=dict(
            identifier=ind_i, project=project, is_public=is_public,
            formula=stoich_to_form_dict.get(stoich, stoich),
            data=dict(dH=dH)
            )
        )

# row_i = df_dft.iloc[0]

IrO3
IrO2
IrO2
IrO2
IrO2
IrO2
IrO2
IrO2
IrO2
IrO2


In [12]:
dH

'1.231027411289104 eV'

In [10]:
if True:
    contribs = []
    for key, val in contributions.items():
        contribs.append(val["contrib"])

    created = client.contributions.create_entries(
        contributions=contribs).result()

    created.keys()
    cid_tmp = created["data"][0]["id"]

In [11]:
assert False

AssertionError: 

# Add Structures

In [ ]:
row_i = df_dft.iloc[0]

atoms = row_i.atoms

structure = AseAtomsAdaptor.get_structure(atoms)

In [ ]:
# structure = Structure.from_file('Fe3O4.cif')

# for i in created:

sdct = dict(contribution=cid_tmp, name='Fe3O4',
    label="temp_label",
    # name=
    )

# 'label': '2019/05/21', 'name': comp

sdct.update(structure.as_dict())


# sid = client.structures.create_entries(structure=[sdct]).result()['id']
sid = client.structures.create_entries(structures=[sdct])

In [ ]:
# identifier = 'mp-1002'
# cid = client.contributions.create_entry(contribution={
#     'project': project, 'identifier': identifier,
#     'data': {'E': '3.33 eV', 'E|V': {'a': 1, 'b': '3 cm'}}
# }).result()['id']

# MISC
---

# Extracting data with `get_entries` methods

In [ ]:
# identifier = "mp-1234"

# client.contributions.get_entries(
#     project=project,
#     identifier=identifier,
#     # _fields=["formula"],
#     ).result()
#     # ).result()['data']

# # client.projects.get_entries?

In [ ]:
# identifier = 'mp-1002-temp'
# cid = client.contributions.create_entry(contribution={
#     'project': project, 'identifier': identifier,
#     'data': {'E': '3.33 eV', 'E|V': {'a': 1, 'b': '3 cm'}}
# }).result()['id']

In [ ]:
# is_public = True

# # dir(client.contributions)
# # client.contributions.create_entries?

# contributions = {
#     "mp-001": {
#         "contrib": {
#             "identifier": "mp-001",
#             "project": project, "is_public": is_public,
#             "data": {"temp0": 0, "temp1": 1},
#             }
#         },

#     "mp-002": {
#         "contrib": {
#             "identifier": "mp-002",
#             "project": project, "is_public": is_public,
#             "data": {"temp0": 8, "temp1": 9},
#             }
#         },

# #     "mp-002": ,
# #     "mp-003": ,
#     }